In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import math

In [140]:
def multivariate_univariate_single_step(sequence,window_size):
    x, y = list(), list()
    for i in range(len(sequence)):
    # find the end of this pattern
        end_ix = i + window_size
        # check if we are beyond the sequence
        if end_ix > len(sequence):
            break
    # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix,:-1], sequence[end_ix-1,-1]
        x.append(seq_x)
        y.append(seq_y)
    return np.array(x), np.array(y)



def multivariate_univariate_multi_step(sequence,window_size,n_multistep):
    x, y = list(), list()
    for i in range(len(sequence)):
    # find the end of this pattern
        end_ix = i + window_size
        out_ix = end_ix + n_multistep -1
        # check if we are beyond the sequence
        if out_ix > len(sequence):
            break
    # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix,:-1], sequence[end_ix-1:out_ix,-1]
        x.append(seq_x)
        y.append(seq_y)
    return np.array(x), np.array(y)

In [117]:
split_ratio = 0.70
num_epochs = 20
window_size = 4
n_step = 1

#seed
torch.manual_seed(123)

In [118]:
in_seq1 = np.array([x for x in range(0, 300, 10)])
in_seq2 = np.array([x for x in range(5, 305, 10)])
out_seq = np.array([in_seq1[i] + in_seq2[i] for i in range(len(in_seq1))])


# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
dataset

array([[  0,   5,   5],
       [ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185],
       [100, 105, 205],
       [110, 115, 225],
       [120, 125, 245],
       [130, 135, 265],
       [140, 145, 285],
       [150, 155, 305],
       [160, 165, 325],
       [170, 175, 345],
       [180, 185, 365],
       [190, 195, 385],
       [200, 205, 405],
       [210, 215, 425],
       [220, 225, 445],
       [230, 235, 465],
       [240, 245, 485],
       [250, 255, 505],
       [260, 265, 525],
       [270, 275, 545],
       [280, 285, 565],
       [290, 295, 585]])

In [119]:
split_data = round(len(dataset)*split_ratio)
split_data

21

In [120]:
#split data by indexing 
train_data = dataset[:split_data]
test_data = dataset[split_data:]
print("train_data_shape")
print(train_data.shape)
print("test_data_shape")
print(test_data.shape)

train_data_shape
(21, 3)
test_data_shape
(9, 3)


In [121]:
scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(train_data.reshape(-1, 1))
test_data_normalized = scaler.fit_transform(test_data.reshape(-1, 1))

In [122]:
#transform after scaling
train_data_normalized = train_data_normalized.reshape(train_data.shape[0],train_data.shape[1])
print("test_data_normalized"+str(train_data_normalized.shape))

test_data_normalized = test_data_normalized.reshape(test_data.shape[0],test_data.shape[1])
print("test_data_normalized"+str(test_data_normalized.shape))

test_data_normalized(21, 3)
test_data_normalized(9, 3)


In [123]:
trainX ,trainY =  multivariate_univariate_single_step(train_data_normalized,window_size)
testX , testY = multivariate_univariate_single_step(test_data_normalized,window_size)
print(f"trainX shape:{trainX.shape} trainY shape:{trainY.shape}\n")
print(f"testX shape:{testX.shape} testY shape:{testY.shape}")

trainX shape:(18, 4, 2) trainY shape:(18,)

testX shape:(6, 4, 2) testY shape:(6,)


In [124]:
# make training and test sets in torch
trainX = torch.from_numpy(trainX).type(torch.Tensor)
trainY = torch.from_numpy(trainY).type(torch.Tensor)
testX = torch.from_numpy(testX).type(torch.Tensor)
testY = torch.from_numpy(testY).type(torch.Tensor)

In [125]:
#3D Data Preparation
trainX = torch.reshape(trainX,(trainX.shape[0],trainX.shape[1],trainX.shape[2]))
trainY = torch.reshape(trainY,(trainY.shape[0],n_step))
testX = torch.reshape(testX,(testX.shape[0],trainX.shape[1],trainX.shape[2]))
testY = torch.reshape(testY,(testY.shape[0],n_step))

# Vanila LSTM

In [126]:
class LSTM(nn.Module):

        def __init__(self, n_feature, hidden_dim, num_layers, output_dim):
            super(LSTM, self).__init__()

            self.n_feature = n_feature
            # Hidden dimensions
            self.hidden_dim = hidden_dim

            # Number of hidden layers
            self.num_layers = num_layers

            # Building your LSTM
            # batch_first=True causes input/output tensors to be of shape
            # (batch_dim, seq_dim, feature_dim)
            self.lstm = nn.LSTM(n_feature, hidden_dim, num_layers, batch_first=True)

            # Readout layer
            self.fc = nn.Linear(hidden_dim, output_dim)


        def forward(self, x):
            # Initialize hidden state with zeros
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

            # Initialize cell state
            c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

            # One time step
            # We need to detach as we are doing truncated backpropagation through time (BPTT)
            # If we don't, we'll backprop all the way to the start even after going through another batch
            out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

            # Index hidden state of last time step
            # out.size() --> 100, 28, 100
            # out[:, -1, :] --> 100, 100 --> just want last time step hidden states!
            out = self.fc(out[:, -1, :])
            # out.size() --> 100, 10
            return out

## Bidirectional LSTM

In [127]:
class BidirectionalLSTM(nn.Module):

    def __init__(self, n_feature, hidden_dim, num_layers, output_dim):
        super(BidirectionalLSTM, self).__init__()

        self.n_feature = n_feature
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.num_layers = num_layers

        # Building your LSTM
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(n_feature, hidden_dim, num_layers, batch_first=True,bidirectional=True)

        # Readout layer *2 for bidirectional LSTM
        self.fc = nn.Linear(hidden_dim*2, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_dim).requires_grad_()

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        out = self.fc(out[:, -1, :])
        
        return out

In [128]:
#Arguments for LSTM model
hidden_dim = 10
number_of_time_series = trainX.shape[2] 

#1 for vanila LSTM , >1 is mean stacked LSTM
num_layers = 1

#Vanila , Stacked LSTM
# model = LSTM(n_feature=number_of_time_series, hidden_dim=hidden_dim, output_dim=n_step, num_layers=num_layers)

#Bidirectional LSTM
model = BidirectionalLSTM(n_feature=number_of_time_series, hidden_dim=hidden_dim, output_dim=n_step, num_layers=num_layers)

In [129]:
number_of_time_series

2

In [130]:
#loss function 
loss_fn = torch.nn.MSELoss()

#optimiser
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [131]:
for t in range(num_epochs):
    # Initialise hidden state
#     Don't do this if you want your LSTM to be stateful
#     model.hidden = model.init_hidden()

    # Forward pass
    y_train_pred = model(trainX)
    
    #Reshape to perform MSE 
    y_train_pred=torch.reshape(y_train_pred,(trainY.shape[0],trainY.shape[1]))

    loss = loss_fn(y_train_pred, trainY)
    print("Epoch ", t, "MSE: ", loss.item())

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()

Epoch  0 MSE:  0.41715627908706665
Epoch  1 MSE:  0.3755790591239929
Epoch  2 MSE:  0.3406274616718292
Epoch  3 MSE:  0.31210410594940186
Epoch  4 MSE:  0.2894130349159241
Epoch  5 MSE:  0.2716890573501587
Epoch  6 MSE:  0.25820520520210266
Epoch  7 MSE:  0.24816280603408813
Epoch  8 MSE:  0.24015048146247864
Epoch  9 MSE:  0.23214925825595856
Epoch  10 MSE:  0.22232572734355927
Epoch  11 MSE:  0.20983698964118958
Epoch  12 MSE:  0.19493837654590607
Epoch  13 MSE:  0.17857730388641357
Epoch  14 MSE:  0.16195949912071228
Epoch  15 MSE:  0.146148219704628
Epoch  16 MSE:  0.13169163465499878
Epoch  17 MSE:  0.11832424998283386
Epoch  18 MSE:  0.10502355545759201
Epoch  19 MSE:  0.09063974767923355


In [132]:
# make predictions
y_test_pred = model(testX)

#Reshape to original data
y_train_pred = torch.reshape(y_train_pred,(y_train_pred.shape[0],y_train_pred.shape[1]))
trainY = torch.reshape(trainY,(trainY.shape[0],trainY.shape[1]))
y_test_pred = torch.reshape(y_test_pred,(y_test_pred.shape[0],y_test_pred.shape[1]))
testY = torch.reshape(testY,(testY.shape[0],testY.shape[1]))

In [133]:
#Invert predictions
y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
y_train = scaler.inverse_transform(trainY.detach().numpy())
y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
y_test = scaler.inverse_transform(testY.detach().numpy())

In [134]:
print("y-test\t\t\t\ty-predict")
for i in range(len(y_test_pred)):
    print(f"{y_test[i]}\t\t{y_test_pred[i]}")

y-test				y-predict
[484.99997]		[343.20303]
[505.]		[350.80276]
[525.]		[358.8228]
[545.]		[367.2727]
[565.]		[376.1521]
[585.]		[385.44693]


In [135]:
print(f"y_test_shape : {y_test.shape}")
print(f"y_test_pred_shape : {y_test_pred.shape}")

y_test_shape : (6, 1)
y_test_pred_shape : (6, 1)


In [136]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train[:,0], y_train_pred[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 56.45 RMSE
Test Score: 172.52 RMSE


## Exercise for Multivariate (Solution)